# Query Latency & Cost

In [ ]:
import concurrent.futures
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns

sns.set_theme(context="notebook", style="white", font="Times New Roman", font_scale=1.5)
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

url_prefix = "https://lfnn6pvnsmonpwho3pp3os6eb40fxqyo.lambda-url.us-east-1.on.aws/?compiler_name=kTpch&query_plan=kTpchQ6&scale_factor=kSf"
url_suffix = "&storage_prefix=systemDemo"
scale_factors = [1, 10, 100, 1000]
runtimes_ms = []
costs_cent = []

def fetch_runtime(scale_factor):
    url = url_prefix + str(scale_factor) + url_suffix
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        runtime_ms = data.get("execution_statistics", {}).get("execution_runtime_ms", None)
        cost_cent = data.get("execution_statistics", {}).get("execution_cost_cent", None)
        return runtime_ms, cost_cent
    else:
        print(f"Error for scale_factor {scale_factor}: {response.status_code}, {response.text}")
        return None

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(fetch_runtime, sf): sf for sf in scale_factors}
    for future in concurrent.futures.as_completed(futures):
        runtime, cost = future.result()
        if runtime is not None:
            runtimes_ms.append(runtime)
        if cost is not None:
            costs_cent.append(cost)

df_runtimes = pd.DataFrame({'scale_factor': scale_factors, 'runtime_ms': list(runtimes_ms), 'cost_cent': list(costs_cent)}, columns=['scale_factor', 'runtime_ms', 'cost_cent'])
df_runtimes['runtime_s'] = df_runtimes['runtime_ms'] / 1000
df_runtimes = df_runtimes.sort_values('scale_factor')
print(df_runtimes)

x = np.arange(len(scale_factors))
width = 0.4

runtime_ticks = np.arange(0, 15, 3)
cost_ticks = np.arange(0, 7.5, 1.5)

fig, ax1 = plt.subplots()

ax1.bar(x - width/2, df_runtimes['runtime_s'], width, color='#0173b2')
ax1.set_xlabel("Scale Factor")
ax1.set_ylabel("Query Latency [s]")
ax1.tick_params(axis="y")
ax1.set_xticks(x)
ax1.set_xticklabels(scale_factors)
ax1.set_yticks(runtime_ticks)

ax2 = ax1.twinx()
ax2.bar(x + width/2, df_runtimes['cost_cent'], width, color='#d55e00')
ax2.set_ylabel("Query Cost [¢]")
ax2.tick_params(axis="y")
ax2.set_yscale('log')

fig.tight_layout()
plt.show()